In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00


# Spam SNS classification

메일(텍스트)이 spam 메일인지 아닌지를 판별하는 문장 분류 태스크  
dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset  
4000개까지 train으로 사용  

#Dataset 구축  
dataset에서는 __ getitem __을 통해 한개의 데이터를 모델의 입력형태에 맞추어 반환한다.  
이번 코드에서는 자연어 문장을 모델에 입력하기 위해 tokenization과 vocab dictionary에 따른 index로의 변환을 진행한다.  
또한 label의 ham/spam에 따라 0/1을 label로 변환한다.  

주요 class, method:  
torchtext.data.utils.get_tokenizer: torchtext에서 제공하는 tokenizer 문법에 따른 tokenization을 수행하는 class 반환  
torchtext.vocab.build_vocab_from_iterator:  내 학습데이터에 대한 모든 단어를 입력하면 각 단어에 한개씩 index를 부여한 vocab dictionary 반환  
**huggingface.tokenizer:** 사전에 학습된 tokenization을 불러오는 huggingface class  
https://huggingface.co/docs/transformers/main_classes/tokenizer


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,BertTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#dataset load
data_path = "/content/drive/MyDrive/nlp-open-tutorial/4일차_배포/dataset/spam.csv"
data_df = pd.read_csv(data_path, encoding = "ISO-8859-1")

#train test split
data_df = data_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
train_df = data_df.loc[:4000,:].reset_index()
test_df = data_df.loc[4000:,:].reset_index()

#tokenizer load
# tokenizer = get_tokenizer("basic_english")

#build vocab dictionary
# vocab_dict = build_vocab_from_iterator(list(map(tokenizer, data_df)))

#huggingface tokenizer load
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class myDataset(Dataset):
  def __init__(self, df, tokenizer) -> None:
      super().__init__()
      self.df = df
      self.tokenizer = tokenizer

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      data = self.df.loc[index, "v2"]
      target = self.df.loc[index, "v1"]

      #data tokenization
      data = self.tokenizer(data, max_length = 100, padding="max_length", truncation=True)['input_ids']

      #labeling
      label = 1 if target=="spam" else 0

      return torch.IntTensor(data), label

#train, test dataset 선언
train_dataset = myDataset(train_df, tokenizer)
test_dataset = myDataset(test_df, tokenizer)

#train, test dataloader 선언
batch_size = 100
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

#잘 실행되는지 확인
for i in train_dataset:
  print(i[0].shape)
  print(i[1])
  break

for i in train_dataloader:
  print(i[0].shape)
  print(i[1].shape)
  break

torch.Size([100])
0
torch.Size([100, 100])
torch.Size([100])


#Model 구축
RNN/LSTM을 통해 text를 classification하는 model 선언
1. token의 index를 입력으로 받고 word embedding을 결과로 반환하는 nn.Embedding 사용 (128 차원의 embedding)
2. LSTM을 통해 모든 token을 입력 (128 차원의 hidden state)
3. many-to-one구조를 가지기 때문에 LSTM의 결과중 마지막 cell에 대한 결과만을 사용하여 nn.linear를통해 classification

입력으로는 문장을 tokenziation과 indexing한게 입력으로 들어오기 때문에(batch, sequance length)형태의 입력  
이후 nn.Embedding을 거치면서 각 단어의 벡터가 생성되기 때문에(batch, sequance length, hidden size)의 형태로 nn.LSTM에 입력됨

주요 obejct:  
**nn.Embedding:**https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html  
**nn.LSTM:**https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html  
**nn.RNN:**https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [ ]:
from torch import nn

class myModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.embedding = nn.Embedding(50000, 128)
        self.lstm = nn.LSTM(128, 128)
        self.linear = nn.Linear(128, 2)

    def forward(self, x):
        x = self.embedding(x)
        x, s = self.lstm(x)
        x = self.linear(x[:,-1,:])
        return x

#Model 선언
model = myModel()

#잘 실행되는지 확인
for i in train_dataloader:
  data = i[0]
  label = i[1]
  print(data.shape, label.shape)
  data = model(data)
  print(data.shape)
  break

torch.Size([100, 100]) torch.Size([100])
torch.Size([100, 2])


#Main

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

#Model 선언
model = myModel().cuda()

#학습을 위한 optimizer와 loss function 설정
optimizer = Adam(params=model.parameters(), lr=0.001)
lf = CrossEntropyLoss()

#100번의 에폭을 실행
for e in range(100):
  print("\nepoch ", e)
  epoch_loss = 0
  train_correct = 0

  #선언한 모델 오브젝트를 학습가능한 상태로 변경
  model.train()

  #모든 학습데이터에 대해서 학습
  for i in train_dataloader:
    #매 배치에 대한 gradient계산 이전에 optimizer에 저장된 이전 batch에 gradient를 삭제(초기화)
    optimizer.zero_grad()

    data = i[0].cuda()
    target = i[1].cuda()

    #결과 도출 및 정답수 연산
    output = model(data)

    pred_label = torch.argmax(output, dim=-1)
    train_correct += sum(pred_label == target)

    #loss연산
    loss = lf(output, target)

    #loss backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()

  print(train_correct)
  print("train loss", epoch_loss/len(train_dataloader))
  print("train acc", train_correct/len(train_dataset))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0
  test_correct = 0

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in test_dataloader:
      data = i[0].cuda()
      target = i[1].cuda()

      output = model(data)

      loss = lf(output, target)
      pred_label = torch.argmax(output, dim=-1)
      test_correct += sum(pred_label==target)
      test_loss += loss.item()

  print("test loss", test_loss/len(test_dataloader))
  print("test acc", test_correct/len(test_dataset))




epoch  0
tensor(3466, device='cuda:0')
train loss 0.4057011233597267
train acc tensor(0.8663, device='cuda:0')
test loss 0.4014686793088913
test acc tensor(0.8651, device='cuda:0')

epoch  1
tensor(3466, device='cuda:0')
train loss 0.39095816016197205
train acc tensor(0.8663, device='cuda:0')
test loss 0.399105167016387
test acc tensor(0.8651, device='cuda:0')

epoch  2
tensor(3466, device='cuda:0')
train loss 0.390461398334038
train acc tensor(0.8663, device='cuda:0')
test loss 0.39539206586778164
test acc tensor(0.8651, device='cuda:0')

epoch  3
tensor(3466, device='cuda:0')
train loss 0.39041187159898805
train acc tensor(0.8663, device='cuda:0')
test loss 0.39580465108156204
test acc tensor(0.8651, device='cuda:0')

epoch  4
tensor(3466, device='cuda:0')
train loss 0.43285836679179496
train acc tensor(0.8663, device='cuda:0')
test loss 0.39414174295961857
test acc tensor(0.8651, device='cuda:0')

epoch  5
tensor(3466, device='cuda:0')
train loss 0.390869701780924
train acc tensor(